In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json

load_dotenv()

client_open_ai = OpenAI(api_key=os.environ.get("OPEN_AI_API_KEY"))

In [2]:
from opensearchpy import OpenSearch

def get_open_search_client():
    host = 'localhost'
    port = 9200

    auth = ('admin', 'admin')

    client_open_search = OpenSearch(
        hosts = [{'host': host, 'port': port}],
        http_auth = auth,
        use_ssl = True,
        verify_certs = False
    )
    return client_open_search

In [3]:
def open_search_quary(search_body, index_name, client_open_search, size, from_where: int=None):
    response = client_open_search.search(index=index_name, body=search_body, size=size, from_=from_where)
    if len(response['hits'])>0:
        return response['hits']['hits']
    else:
        print('There is no hits in open search DB.')
        return None

In [4]:
def query_with_open_AI(paper_detail):
  completion = client_open_ai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": 'Generate 4 QA pairs for these information, inforamtion is seperated with \\n'},
      {"role": "user", "content": f"{paper_detail['Title']}\n{paper_detail['Authors']}\n{paper_detail['Abstract']}\n{paper_detail['ArticleDate']}"},
      {"role": "system", "content": 'Return a dictionary with question and answer as keyword, the values of them are lists of generated results, like {"question":["{question1}"...],"answer":["{answer1}"...]}'},
    ],
  )
  print(f"{completion.choices[0].message.content}\n")

  return json.loads(completion.choices[0].message.content)



In [5]:
import time
def generate_qa_pairs(paper_detail):
    data = []
    for i in paper_detail:
        paper = i['_source']
        result = query_with_open_AI(paper)
        
        questions_list=result['question']
        answers_list=result['answer']

        article_title = paper['Title']
        PMID = paper['PMID']
        author = ', '.join(paper['Authors'])
        article_date = paper['ArticleDate']
        context = paper['Abstract']
        final_result = [[qu, an, article_title, PMID, author, context ,article_date] for qu, an in zip(questions_list, answers_list)]
        data+=final_result
        # Avoid OpenAI blocking
        time.sleep(10)
    return data

In [6]:
import csv
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

index_name = 'pubmed_intelligence'

saving_file_name = 'reformatting_testing_qa.csv'

header = ['question', 'answer', 'article_title', 'PMID', 'author', 'context', 'article_date']

client_open_search = get_open_search_client()

search_body = {
    "query": {
        "match": {
            "_index": index_name
        }
    },
}

query_result = open_search_quary(search_body,index_name,client_open_search, 5, 2)


f:\Heidelberg\semester_5\INLPT\Final Project\.venv\lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [7]:
if query_result is not None:

    data = generate_qa_pairs(query_result)

    with open(saving_file_name, 'a+') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(header)
        # write the data
        writer.writerows(data)

{
"question": [
"What is the focus of the study on new-onset benign epilepsy with centrotemporal spikes (BECTS)?",
"How many children with BECTS were enrolled in the study?",
"What were the significant findings in the study related to language?",
"What can be inferred about the neuropsychological profile of children with new-onset BECTS based on the study?"
],
"answer": [
"The focus of the study was on the neuropsychological profile, specifically on executive functions, of children with new-onset benign epilepsy with centrotemporal spikes (BECTS).",
"Fifteen children with BECTS were enrolled in the study.",
"The significant findings related to language in the study included difficulties in color naming, spoonerism, and phonemic synthesis in children with BECTS.",
"The study suggests that children with new-onset BECTS may demonstrate neuropsychological dysfunctions, particularly affecting executive attention, despite normal IQ, low frequency of NREM sleep discharges, and absence of drug